<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/plays_data_extraction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
Data_basic = pd.DataFrame()
Data_basic = pd.DataFrame(columns=[
    'GameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2015,2023))

In [50]:


def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    rows=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      rows.append(df2['id'][0])
      rows.append(season)
      rows.append(types)
      rows.append(weeks)
      rows.append(df2['competitors'][0][0]['id'])
      rows.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          rows.append('Home')
        else:
          rows.append('Away')
      except KeyError:
        rows.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = rows

     


In [51]:
infos(2017,2,2)

In [5]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

                                                $ref         id  \
0  http://sports.core.api.espn.com/v2/sports/foot...  400791485   

                                 uid               date  attendance  \
0  s:20~l:28~e:400791485~c:400791485  2015-09-11T00:30Z       66829   

   necessary  timeValid  neutralSite  divisionCompetition  \
0      False       True        False                False   

   conferenceCompetition  ...  \
0                  False  ...   

                                   powerIndexes.$ref  \
0  http://sports.core.api.espn.com/v2/sports/foot...   

   format.regulation.periods  format.regulation.displayName  \
0                          4                        Quarter   

   format.regulation.slug  format.regulation.clock  format.overtime.periods  \
0                 quarter                    900.0                        1   

   format.overtime.displayName  format.overtime.slug  format.overtime.clock  \
0                 sudden-death          sudden-death  

KeyboardInterrupt: ignored

In [40]:
Data_basic

,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,400951620,2017,2,2,4,34,Away
1,400951632,2017,2,2,11,22,Away
2,400951636,2017,2,2,12,21,Home
3,400951639,2017,2,2,18,17,Away
4,400951643,2017,2,2,23,16,Home
5,400951645,2017,2,2,27,3,Home
6,400951630,2017,2,2,29,2,Home
7,400951635,2017,2,2,30,10,Away
8,400951626,2017,2,2,33,5,Home
9,400951669,2017,2,2,13,20,Home


In [ ]:
playid_df = pd.DataFrame()
playid_df = pd.DataFrame(columns=['gameID','playID'])

In [125]:
def playid(GameID):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
    response_drive = r.get(url)
    drive_data = response_drive.json()
    drive_df = json_normalize(drive_data['items'])
    for index, row in drive_df.iterrows():
      for detail in range(len(row['plays.items'])):
        playid = row['plays.items'][detail]['id']
        print(playid)
        awayScore = row['plays.items'][detail]['awayScore']
        homeScore = row['plays.items'][detail]['homeScore']
        scoreValue = row['plays.items'][detail]['scoreValue']
        statYardage = row['plays.items'][detail]['statYardage']
        clock = datetime.timedelta(seconds=row['plays.items'][detail]['clock']['value'])
        type_id = row['plays.items'][detail]['type']['id']
        type_text = row['plays.items'][detail]['type']['text']
        print(type_text)
        try:
          print(re.search(r"teams/(\d+)", row['plays.items'][detail]['start']['team']['$ref']).group(1))
        except TypeError:
          print('NoTeam')
        except KeyError:
          print('NoTeam')
        start_down = row['plays.items'][detail]['start']['down']
        start_distance = row['plays.items'][detail]['start']['distance']
        start_yardlinie = row['plays.items'][detail]['start']['yardLine']
        start_yardsToEndzone = row['plays.items'][detail]['start']['yardsToEndzone']
        end_down = row['plays.items'][detail]['end']['down']

      #print(row)

In [126]:
playid('401326635')

40132663540
Kickoff
12
40132663555
Rush
4
40132663576
Pass Reception
4
401326635100
Pass Incompletion
4
401326635122
Punt
4
401326635141
Rush
12
401326635162
Pass Reception
12
401326635186
Pass Reception
12
401326635210
Rush
12
401326635231
Rush
12
401326635252
Timeout
12
401326635269
Rush
12
401326635316
Pass Reception
12
401326635340
Rush
12
401326635371
Rush
12
401326635392
Pass Reception
12
401326635421
Passing Touchdown
12
401326635462
Kickoff
12
401326635477
Rush
4
401326635498
Pass Reception
4
401326635522
Rush
4
401326635543
Rush
4
401326635566
Pass Reception
4
401326635590
Pass Reception
4
401326635614
Pass Incompletion
4
401326635638
Pass Reception
4
401326635662
Pass Reception
4
401326635686
Rush
4
401326635712
Pass Incompletion
4
401326635743
Pass Incompletion
4
401326635765
Pass Incompletion
4
401326635787
Field Goal Good
4
401326635806
Kickoff
4
401326635821
Pass Reception
12
401326635845
End Period
NoTeam
401326635864
Rush
12
401326635885
Pass Incompletion
12
40132663590

In [124]:
for games in Data_basic['GameID']:
  playid(games)

40095162037
Kickoff
4
40095162052
Rush
34
40095162073
Pass Incompletion
34
400951620102
Pass Incompletion
34
400951620124
Punt
34
400951620149
Rush
4
400951620170
Pass Reception
4
400951620214
Pass Reception
4
400951620238
Sack
4
400951620268
Penalty
4
400951620299
Punt
4
400951620321
Rush
34
400951620353
Pass Reception
34
400951620377
Rush
34
400951620398
Punt
34
400951620423
Penalty
4
400951620455
Pass Incompletion
4
400951620481
Rush
4
400951620502
Pass Reception
4
400951620526
Punt
4
400951620549
Pass Reception
34
400951620573
Rush
34
400951620594
Rush
34
400951620615
Timeout
34
400951620632
Pass Incompletion
34
400951620654
Sack
34
400951620673
Punt
34
400951620698
Penalty
4
400951620733
Fumble Recovery (Opponent)
4
400951620765
Rush
34
400951620786
Rush
34
400951620807
Penalty
34
400951620830
Rush
34
400951620851
Rush
34
400951620872
Pass Incompletion
34
400951620894
Field Goal Good
34
400951620913
Kickoff
34
400951620947
Pass Reception
4
400951620971
Pass Reception
4
40095162099

KeyboardInterrupt: ignored

In [ ]:
playid_df

,gameID,playID
0,400791485,40079148536
1,400791485,40079148551
2,400791485,40079148572
3,400791485,400791485101
4,400791485,400791485122
...,...,...
284571,401326460,4013264603708
284572,401326460,4013264603729
284573,401326460,4013264603760
284574,401326460,4013264603779


In [ ]:
playid_df.to_csv('/content/drive/MyDrive/gameid_playid_data.csv', index=False)